In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_parquet("data.parquet")
print(df.head(1000))

                     banknifty     nifty  tte
time                                         
2021-01-01 09:15:00   0.286058  0.199729   27
2021-01-01 09:16:00   0.285381  0.200433   27
2021-01-01 09:17:00   0.284233  0.200004   27
2021-01-01 09:18:00   0.286104  0.199860   27
2021-01-01 09:19:00   0.285539  0.198951   27
...                        ...       ...  ...
2021-01-02 01:50:00   0.255651  0.178049   27
2021-01-02 01:51:00   0.255651  0.178049   27
2021-01-02 01:52:00   0.255651  0.178049   27
2021-01-02 01:53:00   0.255651  0.178049   27
2021-01-02 01:54:00   0.255651  0.178049   27

[1000 rows x 3 columns]


In [3]:
df = df.reset_index()

In [4]:
print(df.describe())

           banknifty          nifty            tte
count  689012.000000  688512.000000  690512.000000
mean        0.250156       0.179343      15.671884
std         0.057702       0.046139       8.664597
min         0.152732       0.089355       2.000000
25%         0.205307       0.140375       8.000000
50%         0.243977       0.181210      15.000000
75%         0.295293       0.215635      23.000000
max         0.460619       0.306952      31.000000


In [5]:
df['time'] = pd.to_datetime(df['time'])

In [6]:
df['banknifty'].fillna(method='ffill', inplace=True)
df['nifty'].fillna(method='ffill', inplace=True)

In [7]:
df = df[df['time'].dt.dayofweek < 5]
print(df.shape)

(494672, 4)


In [8]:
import datetime
start_time = datetime.time(9, 15)
end_time = datetime.time(15, 30)

In [9]:
df = df[df['time'].dt.time.between(start_time, end_time)]
print(df.shape)

(129720, 4)


### Grid Search for Best z-Score Entry/Exit (25% Train / 75% Test)

**Goal:**  
Tune `entry_z` and `exit_z` thresholds for a mean-reversion strategy on the BankNifty–Nifty spread.

---

### Workflow
1. **Preprocess**
   - Compute `spread = banknifty - nifty`.
   - Rolling mean/std → `zscore = (spread - μ)/σ`.
   - `PLValue = spread * (TTE ** 0.7)` for PnL.
   - Drop NaNs.

2. **Split**
   - 25% of data → **train**.
   - 75% of data → **test**.

3. **Train (Grid Search)**
   - Try all `entry_z ∈ [1.0 … 3.5]`, `exit_z ∈ {0.0, 0.25, 0.5, 0.75, 1.0}`.
   - Run simulator, calculate metrics.
   - Pick combo with **max train PnL** (tie-break Sharpe).

4. **Test**
   - Evaluate chosen params on unseen 75%.

---

### Trading Logic
- **Enter LONG**: z crosses **down** through `-entry_z`.
- **Enter SHORT**: z crosses **up** through `+entry_z`.
- **Exit toward 0**:  
  - Long → exit when z crosses up to `-exit_z`.  
  - Short → exit when z crosses down to `+exit_z`.  
- Time stop: min = `min_horizon`, max = `max_horizon`.

---

### Metrics
- **PnL**: cumulative profit/loss.
- **Sharpe**: mean/std of per-trade returns (×√252).
- **Max Drawdown**: worst peak-to-trough.
- **Winrate**: % of positive trades.

---

### Output
- Table by `window` (30, 120, 375) showing:
  - Best `entry_z`, `exit_z`
  - Train metrics
  - Test metrics

In [13]:
# ==============================================
# Best entry/exit z by window (25% train split)
# ==============================================
import numpy as np
import pandas as pd

# -------------------------
# EDITABLE GRIDS & SETTINGS
# -------------------------
windows        = [30, 120, 375]
entry_grid     = np.arange(1.0, 3.51, 0.25)     # try 1.0 … 3.5
exit_grid      = np.array([0.0, 0.25, 0.5, 0.75, 1.0])   # toward 0
min_horizon    = 30
max_horizon    = 5 * 375
cooldown_bars  = 0
roundtrip_cost = 0.0   # if you want to include costs, set > 0

# --------------------------------
# PREP (expects df with these cols)
# --------------------------------
df = df.sort_values('time').drop_duplicates('time').copy()
df['time']      = pd.to_datetime(df['time'])
df['banknifty'] = df['banknifty'].astype(float)
df['nifty']     = df['nifty'].astype(float)
df['tte']       = df['tte'].astype(float)

# -------------------------
# HELPERS
# -------------------------
def make_df_with_z(df0, window):
    d = df0.copy()
    d['spread'] = d['banknifty'] - d['nifty']
    roll = d['spread'].rolling(window, min_periods=window)
    mu   = roll.mean()
    sd   = roll.std(ddof=0).replace(0, np.nan)
    d['zscore'] = (d['spread'] - mu) / sd
    # assignment PL value
    d['PLValue'] = d['spread'] * (d['tte'] ** 0.7)
    d = d.dropna(subset=['zscore','tte']).reset_index(drop=True)
    return d

def run_trading_sim(d, entry_z, exit_z, min_horizon=30, max_horizon=1950, cooldown=0, cost=0.0):
    """
    Long when z crosses DOWN through -entry_z; exit when it reverts UP to -exit_z.
    Short when z crosses UP through +entry_z; exit when it reverts DOWN to +exit_z.
    (Crossing logic; exits toward 0; optional cooldown.)
    """
    trades = []
    pos = None
    cooldown_left = 0

    for i in range(1, len(d)):
        z_prev, z = d['zscore'].iat[i-1], d['zscore'].iat[i]

        if cooldown_left > 0:
            cooldown_left -= 1

        # ENTRY
        if pos is None and cooldown_left == 0:
            if (z_prev <=  entry_z) and (z >  entry_z):   # cross up → SHORT
                pos = {'side':'short', 'i0':i}
                continue
            if (z_prev >= -entry_z) and (z < -entry_z):   # cross down → LONG
                pos = {'side':'long',  'i0':i}
                continue

        # EXIT
        if pos is not None:
            hold = i - pos['i0']
            exit_long  = (pos['side']=='long'  and (z_prev < -exit_z) and (z >= -exit_z))
            exit_short = (pos['side']=='short' and (z_prev >  exit_z) and (z <=  exit_z))
            time_up    = hold >= max_horizon

            if hold >= min_horizon and (exit_long or exit_short or time_up):
                ent = d.iloc[pos['i0']]
                cur = d.iloc[i]
                pnl_ent = ent['PLValue']
                pnl_cur = cur['PLValue']
                pnl     = (pnl_cur - pnl_ent) if pos['side']=='long' else (pnl_ent - pnl_cur)
                pnl    -= cost  # per roundtrip
                trades.append({
                    'side': pos['side'],
                    'entry_time': ent['time'],
                    'exit_time':  cur['time'],
                    'minutes': hold,
                    'entry_z': d['zscore'].iat[pos['i0']],
                    'exit_z':  z,
                    'pnl': float(pnl)
                })
                pos = None
                cooldown_left = cooldown
    return pd.DataFrame(trades)

def evaluate(trades_df):
    if trades_df.empty:
        return {'trades':0, 'pnl':0.0, 'sharpe':np.nan, 'max_dd':np.nan, 'winrate':np.nan}
    pnl_curve = trades_df['pnl'].cumsum()
    rets = trades_df['pnl']
    sharpe = (rets.mean()/rets.std(ddof=0)*np.sqrt(252)) if rets.std(ddof=0)>0 else np.nan
    peak = pnl_curve.cummax()
    max_dd = (pnl_curve - peak).min()
    wins = (trades_df['pnl'] > 0).sum()
    losses = (trades_df['pnl'] < 0).sum()
    winrate = wins/(wins+losses) if (wins+losses)>0 else np.nan
    return {
        'trades': len(trades_df),
        'pnl': float(pnl_curve.iloc[-1]),
        'sharpe': float(sharpe) if np.isfinite(sharpe) else np.nan,
        'max_dd': float(max_dd) if np.isfinite(max_dd) else np.nan,
        'winrate': float(winrate) if np.isfinite(winrate) else np.nan
    }

# -------------------------
# MAIN: per window, train/test
# -------------------------
rows = []
best_params_by_window = {}

for W in windows:
    dW = make_df_with_z(df, W)
    if dW.empty:
        rows.append({'window': W, 'msg': 'not enough data after zscore drop'})
        continue

    # 25% train / 75% test split on the z-scored data
    split = int(0.25 * len(dW))
    d_train = dW.iloc[:split].reset_index(drop=True)
    d_test  = dW.iloc[split:].reset_index(drop=True)

    # grid search on TRAIN
    cand_rows = []
    for ez in entry_grid:
        for xz in exit_grid:
            tr = run_trading_sim(d_train, ez, xz,
                                 min_horizon=min_horizon,
                                 max_horizon=max_horizon,
                                 cooldown=cooldown_bars,
                                 cost=roundtrip_cost)
            stats = evaluate(tr)
            cand_rows.append({'entry_z': ez, 'exit_z': xz, **stats})
    train_grid = pd.DataFrame(cand_rows)

    if train_grid.empty:
        rows.append({'window': W, 'msg': 'no trades on train for any params'})
        continue

    # pick best by TRAIN PnL, tie-break by Sharpe
    train_grid = train_grid.sort_values(['pnl','sharpe'], ascending=[False, False]).reset_index(drop=True)
    best_ez = float(train_grid.loc[0, 'entry_z'])
    best_xz = float(train_grid.loc[0, 'exit_z'])

    # evaluate on TEST with chosen params
    tr_test = run_trading_sim(d_test, best_ez, best_xz,
                              min_horizon=min_horizon,
                              max_horizon=max_horizon,
                              cooldown=cooldown_bars,
                              cost=roundtrip_cost)
    test_stats = evaluate(tr_test)

    # add to rows
    rows.append({
        'window': W,
        'best_entry_z': best_ez,
        'best_exit_z': best_xz,
        'train_trades': int(train_grid.loc[0, 'trades']),
        'train_pnl': float(train_grid.loc[0, 'pnl']),
        'train_sharpe': float(train_grid.loc[0, 'sharpe']) if np.isfinite(train_grid.loc[0, 'sharpe']) else np.nan,
        'train_max_dd': float(train_grid.loc[0, 'max_dd']) if np.isfinite(train_grid.loc[0, 'max_dd']) else np.nan,
        'train_winrate': float(train_grid.loc[0, 'winrate']) if np.isfinite(train_grid.loc[0, 'winrate']) else np.nan,
        'test_trades': test_stats['trades'],
        'test_pnl': test_stats['pnl'],
        'test_sharpe': test_stats['sharpe'],
        'test_max_dd': test_stats['max_dd'],
        'test_winrate': test_stats['winrate']
    })

summary = pd.DataFrame(rows).sort_values('window').reset_index(drop=True)
summary

,window,best_entry_z,best_exit_z,train_trades,train_pnl,train_sharpe,train_max_dd,train_winrate,test_trades,test_pnl,test_sharpe,test_max_dd,test_winrate
0,30,1.00,0.0,597,21.355506,3.770716,-1.644679,0.780570,1959,71.469768,3.391436,-2.202592,0.758550
1,120,1.25,0.0,403,21.990427,4.716610,-1.362327,0.868486,1286,68.056842,3.911443,-2.490735,0.880249
2,375,1.25,0.0,223,31.233581,5.825992,-2.256721,0.843049,713,58.828704,5.369190,-0.925729,0.870968


### ML Optimization of z-Score Entry/Exit (25% Train / 75% Test)

**Goal**  
Use Optuna to learn `entry_z` and `exit_z` per `window` that maximize a **weighted objective** of PnL, Sharpe, and (negative) Drawdown.

---

### Workflow
1. **Preprocess**
   - `spread = banknifty - nifty`
   - Rolling mean/std → `zscore = (spread - μ)/σ`
   - `PLValue = spread * (TTE ** 0.7)`
   - Drop NaNs.

2. **Split**
   - For each `window ∈ {30, 120, 375}`: first **25%** → **train**, last **75%** → **test**.

3. **Optimize (Train)**
   - Optuna samples `entry_z ∈ [1.0, 3.5]`, `exit_z ∈ [0.0, 1.5]`.
   - Simulator runs with crossing logic (no double trades).
   - **Score**: `w_pnl * PnL + w_sharpe * Sharpe − w_dd * |MaxDD|`.

4. **Evaluate (Test)**
   - Re-run simulator on the test slice using the **best train params**.

---

### Trading Rules (Crossing)
- **Enter LONG**: z crosses **down** through `-entry_z`.
- **Enter SHORT**: z crosses **up** through `+entry_z`.
- **Exit toward 0**:
  - Long → exit when z crosses **up** to `-exit_z`.
  - Short → exit when z crosses **down** to `+exit_z`.
- Enforce `min_horizon`, `max_horizon`; skip exit on entry bar; optional `cooldown_bars`.

---

### Metrics & Objective
- **Metrics**: Trades, PnL, Sharpe, Max DD, Winrate.
- **Objective (train)**:  
  `score = w_pnl * PnL + w_sharpe * Sharpe − w_dd * |MaxDD|`
  - Increase `w_pnl` to favor profit, increase `w_sharpe` for efficiency, increase `w_dd` to penalize drawdown.

---

### Output
- Per window: best `entry_z`, `exit_z` (from train), **train** metrics & score, and **test** metrics & score.
- `summary_opt` table for quick comparison.

In [15]:
# ==============================================
# ML OPTIMIZATION OF ENTRY/EXIT Z (per window)
# 25% train / 75% test, using Optuna to maximize
# weighted (PnL, Sharpe, Drawdown)
# ==============================================
import numpy as np
import pandas as pd
import optuna

# -------------------------
# EDITABLE SETTINGS
# -------------------------
windows         = [30, 120, 375]
train_frac      = 0.25

# Objective weights (larger w_pnl -> prioritize PnL)
w_pnl           = 0.35
w_sharpe        = 0.45
w_dd            = 0.20       # penalty weight on |max drawdown|

# Search bounds
entry_z_min, entry_z_max = 1.0, 3.5
exit_z_min,  exit_z_max  = 0.0, 1.5

# Engine controls
min_horizon     = 30
max_horizon     = 5 * 375
cooldown_bars   = 0          # no double-trades: crossing logic + skip exit on entry bar
roundtrip_cost  = 0.0        # no cost as requested
n_trials        = 1200        # Optuna trial budget (raise if you want)

# --------------------------------
# PREP (expects df with these cols)
# --------------------------------
df = df.sort_values('time').drop_duplicates('time').copy()
df['time']      = pd.to_datetime(df['time'])
df['banknifty'] = df['banknifty'].astype(float)
df['nifty']     = df['nifty'].astype(float)
df['tte']       = df['tte'].astype(float)

# -------------------------
# HELPERS
# -------------------------
def make_df_with_z(df0, window):
    d = df0.copy()
    d['spread'] = d['banknifty'] - d['nifty']
    roll = d['spread'].rolling(window, min_periods=window)
    mu   = roll.mean()
    sd   = roll.std(ddof=0).replace(0, np.nan)
    d['zscore']  = (d['spread'] - mu) / sd
    d['PLValue'] = d['spread'] * (d['tte'] ** 0.7)
    d = d.dropna(subset=['zscore','tte']).reset_index(drop=True)
    return d

def run_trading_sim(d, entry_z, exit_z,
                    min_horizon=30, max_horizon=1950, cooldown=0, cost=0.0):
    """
    Long when z crosses DOWN through -entry_z; exit when it reverts UP to -exit_z.
    Short when z crosses UP through +entry_z; exit when it reverts DOWN to +exit_z.
    Crossing logic; exits toward 0; skip exit on the entry bar; optional cooldown.
    """
    trades, pos, cooldown_left = [], None, 0
    for i in range(1, len(d)):
        z_prev, z = d['zscore'].iat[i-1], d['zscore'].iat[i]

        if cooldown_left > 0:
            cooldown_left -= 1

        # ------- ENTRY (CROSS) -------
        if pos is None and cooldown_left == 0:
            # cross UP through +entry_z -> SHORT
            if (z_prev <=  entry_z) and (z >  entry_z):
                pos = {'side':'short', 'i0':i}
                continue  # skip exit check on the same bar
            # cross DOWN through -entry_z -> LONG
            if (z_prev >= -entry_z) and (z < -entry_z):
                pos = {'side':'long', 'i0':i}
                continue

        # ------- EXIT (toward 0) -------
        if pos is not None:
            hold = i - pos['i0']
            exit_long  = (pos['side']=='long'  and (z_prev < -exit_z) and (z >= -exit_z))
            exit_short = (pos['side']=='short' and (z_prev >  exit_z) and (z <=  exit_z))
            time_up    = hold >= max_horizon

            if hold >= min_horizon and (exit_long or exit_short or time_up):
                ent = d.iloc[pos['i0']]
                cur = d.iloc[i]
                pnl = (cur['PLValue'] - ent['PLValue']) if pos['side']=='long' else (ent['PLValue'] - cur['PLValue'])
                pnl -= cost  # per roundtrip
                trades.append({'pnl': float(pnl)})
                pos = None
                cooldown_left = cooldown
    return pd.DataFrame(trades)

def evaluate(trades_df):
    if trades_df.empty:
        return {'trades':0, 'pnl':0.0, 'sharpe':np.nan, 'max_dd':np.nan, 'winrate':np.nan}
    pnl_curve = trades_df['pnl'].cumsum()
    rets = trades_df['pnl']
    sharpe = (rets.mean()/rets.std(ddof=0)*np.sqrt(252)) if rets.std(ddof=0)>0 else np.nan
    peak = pnl_curve.cummax()
    max_dd = (pnl_curve - peak).min()
    wins = (trades_df['pnl'] > 0).sum()
    losses = (trades_df['pnl'] < 0).sum()
    winrate = wins/(wins+losses) if (wins+losses)>0 else np.nan
    return {
        'trades': len(trades_df),
        'pnl': float(pnl_curve.iloc[-1]),
        'sharpe': float(sharpe) if np.isfinite(sharpe) else np.nan,
        'max_dd': float(max_dd) if np.isfinite(max_dd) else np.nan,
        'winrate': float(winrate) if np.isfinite(winrate) else np.nan
    }

def score_from_metrics(m):
    # maximize w_pnl * PnL + w_sharpe * Sharpe - w_dd * |MaxDD|
    dd_mag = abs(m['max_dd']) if np.isfinite(m['max_dd']) else 1e6
    sharpe_term = m['sharpe'] if np.isfinite(m['sharpe']) else 0.0
    return (w_pnl * m['pnl']) + (w_sharpe * sharpe_term) - (w_dd * dd_mag)

# -------------------------
# RUN PER WINDOW
# -------------------------
rows = []
best_params_by_window = {}
all_test_trades = {}

for W in windows:
    dW = make_df_with_z(df, W)
    if dW.empty:
        rows.append({'window':W, 'msg':'insufficient data after z-score drop'})
        continue

    split = int(train_frac * len(dW))
    d_train = dW.iloc[:split].reset_index(drop=True)
    d_test  = dW.iloc[split:].reset_index(drop=True)

    # ----- Optuna Objective on TRAIN -----
    def objective(trial):
        ez = trial.suggest_float('entry_z', entry_z_min, entry_z_max)
        xz = trial.suggest_float('exit_z',  exit_z_min,  exit_z_max)

        tr = run_trading_sim(d_train, ez, xz,
                             min_horizon=min_horizon, max_horizon=max_horizon,
                             cooldown=cooldown_bars, cost=roundtrip_cost)
        met = evaluate(tr)
        return score_from_metrics(met)

    study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=42))
    study.optimize(objective, n_trials=n_trials, show_progress_bar=False)

    best_ez = float(study.best_trial.params['entry_z'])
    best_xz = float(study.best_trial.params['exit_z'])

    # Evaluate on TRAIN (for reporting) and TEST (out-of-sample)
    tr_train = run_trading_sim(d_train, best_ez, best_xz,
                               min_horizon=min_horizon, max_horizon=max_horizon,
                               cooldown=cooldown_bars, cost=roundtrip_cost)
    met_train = evaluate(tr_train)

    tr_test = run_trading_sim(d_test, best_ez, best_xz,
                              min_horizon=min_horizon, max_horizon=max_horizon,
                              cooldown=cooldown_bars, cost=roundtrip_cost)
    met_test = evaluate(tr_test)

    rows.append({
        'window': W,
        'best_entry_z': best_ez,
        'best_exit_z':  best_xz,
        'train_trades': met_train['trades'],
        'train_pnl':    met_train['pnl'],
        'train_sharpe': met_train['sharpe'],
        'train_max_dd': met_train['max_dd'],
        'train_winrate':met_train['winrate'],
        'test_trades':  met_test['trades'],
        'test_pnl':     met_test['pnl'],
        'test_sharpe':  met_test['sharpe'],
        'test_max_dd':  met_test['max_dd'],
        'test_winrate': met_test['winrate'],
        'train_score':  score_from_metrics(met_train),
        'test_score':   score_from_metrics(met_test)
    })

    best_params_by_window[W] = {'entry_z':best_ez, 'exit_z':best_xz, 'train_score':rows[-1]['train_score']}
    all_test_trades[W] = tr_test

summary_opt = pd.DataFrame(rows).sort_values('window').reset_index(drop=True)
summary_opt

[I 2025-09-29 11:29:21,265] A new study created in memory with name: no-name-ca32c406-9641-45a5-bc4d-08279c1843e4
[I 2025-09-29 11:29:21,487] Trial 0 finished with value: 3.5913160356019436 and parameters: {'entry_z': 1.9363502971184063, 'exit_z': 1.4260714596148742}. Best is trial 0 with value: 3.5913160356019436.
[I 2025-09-29 11:29:21,681] Trial 1 finished with value: 6.227312457797125 and parameters: {'entry_z': 2.8299848545285125, 'exit_z': 0.8979877262955549}. Best is trial 1 with value: 6.227312457797125.
[I 2025-09-29 11:29:21,915] Trial 2 finished with value: 8.292707439564907 and parameters: {'entry_z': 1.3900466011060912, 'exit_z': 0.23399178050430397}. Best is trial 2 with value: 8.292707439564907.
[I 2025-09-29 11:29:22,140] Trial 3 finished with value: 5.498128710673796 and parameters: {'entry_z': 1.1452090304204987, 'exit_z': 1.2992642186624028}. Best is trial 2 with value: 8.292707439564907.
[I 2025-09-29 11:29:22,353] Trial 4 finished with value: 6.223766309301551 and 

[I 2025-09-29 11:29:30,590] Trial 40 finished with value: 6.110644453854578 and parameters: {'entry_z': 1.3699644341490471, 'exit_z': 0.6906828332179928}. Best is trial 12 with value: 8.581672157530273.
[I 2025-09-29 11:29:30,804] Trial 41 finished with value: 7.600962093979286 and parameters: {'entry_z': 2.5856527242443503, 'exit_z': 0.2284505223177961}. Best is trial 12 with value: 8.581672157530273.
[I 2025-09-29 11:29:31,050] Trial 42 finished with value: 7.8596275057901845 and parameters: {'entry_z': 1.1337412212500602, 'exit_z': 0.08151754308635835}. Best is trial 12 with value: 8.581672157530273.
[I 2025-09-29 11:29:31,301] Trial 43 finished with value: 7.910341291523967 and parameters: {'entry_z': 1.1212835653155977, 'exit_z': 0.06730370045849005}. Best is trial 12 with value: 8.581672157530273.
[I 2025-09-29 11:29:31,543] Trial 44 finished with value: 5.583332816593157 and parameters: {'entry_z': 1.5335669023010605, 'exit_z': 0.053466149207941564}. Best is trial 12 with value:

[I 2025-09-29 11:29:40,463] Trial 81 finished with value: 8.256568784732993 and parameters: {'entry_z': 1.4366833190391326, 'exit_z': 0.12088347459160428}. Best is trial 74 with value: 9.250441809180778.
[I 2025-09-29 11:29:40,706] Trial 82 finished with value: 9.197073684950244 and parameters: {'entry_z': 1.4055415469785453, 'exit_z': 0.16274339278990985}. Best is trial 74 with value: 9.250441809180778.
[I 2025-09-29 11:29:40,947] Trial 83 finished with value: 8.237110989475088 and parameters: {'entry_z': 1.3722956309924759, 'exit_z': 0.15466682805286952}. Best is trial 74 with value: 9.250441809180778.
[I 2025-09-29 11:29:41,183] Trial 84 finished with value: 5.096483203152097 and parameters: {'entry_z': 1.7887425015966048, 'exit_z': 0.23181547910650907}. Best is trial 74 with value: 9.250441809180778.
[I 2025-09-29 11:29:41,429] Trial 85 finished with value: 7.866593473885842 and parameters: {'entry_z': 1.0782320280016782, 'exit_z': 0.049839356717905965}. Best is trial 74 with value

[I 2025-09-29 11:29:50,424] Trial 122 finished with value: 4.082562486750431 and parameters: {'entry_z': 1.5169616977784515, 'exit_z': 1.3795446901793251}. Best is trial 92 with value: 9.666282503704325.
[I 2025-09-29 11:29:50,667] Trial 123 finished with value: 6.297412449131912 and parameters: {'entry_z': 1.2244673158468489, 'exit_z': 0.13175237009133467}. Best is trial 92 with value: 9.666282503704325.
[I 2025-09-29 11:29:50,914] Trial 124 finished with value: 9.437947709607037 and parameters: {'entry_z': 1.3157496554273014, 'exit_z': 0.0003130347933945855}. Best is trial 92 with value: 9.666282503704325.
[I 2025-09-29 11:29:51,161] Trial 125 finished with value: 10.240211031871011 and parameters: {'entry_z': 1.307947946148091, 'exit_z': 0.026346682249097305}. Best is trial 125 with value: 10.240211031871011.
[I 2025-09-29 11:29:51,405] Trial 126 finished with value: 8.687144066228575 and parameters: {'entry_z': 1.2421849633332676, 'exit_z': 0.040778332817217586}. Best is trial 125 

[I 2025-09-29 11:30:00,051] Trial 162 finished with value: 9.684188648417623 and parameters: {'entry_z': 1.3939567838622369, 'exit_z': 0.032748034862485155}. Best is trial 125 with value: 10.240211031871011.
[I 2025-09-29 11:30:00,295] Trial 163 finished with value: 6.823446231165494 and parameters: {'entry_z': 1.4732801823934405, 'exit_z': 0.06171729529522915}. Best is trial 125 with value: 10.240211031871011.
[I 2025-09-29 11:30:00,539] Trial 164 finished with value: 9.70104190757483 and parameters: {'entry_z': 1.396714755800312, 'exit_z': 0.029796091856741595}. Best is trial 125 with value: 10.240211031871011.
[I 2025-09-29 11:30:00,784] Trial 165 finished with value: 9.623361109933924 and parameters: {'entry_z': 1.3858044734334203, 'exit_z': 0.035037989781789326}. Best is trial 125 with value: 10.240211031871011.
[I 2025-09-29 11:30:01,025] Trial 166 finished with value: 9.660219878468155 and parameters: {'entry_z': 1.3865560402222885, 'exit_z': 0.001868831532733265}. Best is trial

...

[I 2025-09-29 11:42:43,880] Trial 1045 finished with value: 13.293575760056976 and parameters: {'entry_z': 1.121696798844036, 'exit_z': 0.07581087364257724}. Best is trial 831 with value: 14.236793151228433.
[I 2025-09-29 11:42:44,100] Trial 1046 finished with value: 12.620470767643619 and parameters: {'entry_z': 1.1885083475865523, 'exit_z': 0.028135890425264237}. Best is trial 831 with value: 14.236793151228433.
[I 2025-09-29 11:42:44,321] Trial 1047 finished with value: 4.435187162799409 and parameters: {'entry_z': 1.0897169313649981, 'exit_z': 1.1864551354806225}. Best is trial 831 with value: 14.236793151228433.
[I 2025-09-29 11:42:44,544] Trial 1048 finished with value: 11.044388476782867 and parameters: {'entry_z': 1.0004605269722244, 'exit_z': 0.055276047542132604}. Best is trial 831 with value: 14.236793151228433.
[I 2025-09-29 11:42:44,764] Trial 1049 finished with value: 11.846897339201723 and parameters: {'entry_z': 1.291070290769148, 'exit_z': 0.00032495404477245887}. Best

[I 2025-09-29 11:42:52,754] Trial 1085 finished with value: 12.660547457993328 and parameters: {'entry_z': 1.0974488095472767, 'exit_z': 0.03202660564918631}. Best is trial 831 with value: 14.236793151228433.
[I 2025-09-29 11:42:52,976] Trial 1086 finished with value: 11.022872322111628 and parameters: {'entry_z': 1.041275569948745, 'exit_z': 0.06854160558657053}. Best is trial 831 with value: 14.236793151228433.
[I 2025-09-29 11:42:53,199] Trial 1087 finished with value: 11.368959732362223 and parameters: {'entry_z': 1.2789264409537306, 'exit_z': 0.09555447444647064}. Best is trial 831 with value: 14.236793151228433.
[I 2025-09-29 11:42:53,421] Trial 1088 finished with value: 13.668384665007336 and parameters: {'entry_z': 1.12609750496893, 'exit_z': 0.0001089284865965855}. Best is trial 831 with value: 14.236793151228433.
[I 2025-09-29 11:42:53,641] Trial 1089 finished with value: 12.038787579446856 and parameters: {'entry_z': 1.214656843249448, 'exit_z': 0.05625499347728605}. Best is

[I 2025-09-29 11:43:01,664] Trial 1125 finished with value: 11.987779695120677 and parameters: {'entry_z': 1.131892912939734, 'exit_z': 0.05718737938720252}. Best is trial 831 with value: 14.236793151228433.
[I 2025-09-29 11:43:01,884] Trial 1126 finished with value: 11.894874762697269 and parameters: {'entry_z': 1.1970373890380048, 'exit_z': 0.0009570384102961585}. Best is trial 831 with value: 14.236793151228433.
[I 2025-09-29 11:43:02,111] Trial 1127 finished with value: 10.487787712877262 and parameters: {'entry_z': 1.0428341296472898, 'exit_z': 0.12816741448915908}. Best is trial 831 with value: 14.236793151228433.
[I 2025-09-29 11:43:02,338] Trial 1128 finished with value: 12.451791823671488 and parameters: {'entry_z': 1.0986426062832708, 'exit_z': 0.07309598407539358}. Best is trial 831 with value: 14.236793151228433.
[I 2025-09-29 11:43:02,564] Trial 1129 finished with value: 3.977204057872272 and parameters: {'entry_z': 1.1518128190346881, 'exit_z': 0.9923942491591313}. Best i

[I 2025-09-29 11:43:10,602] Trial 1165 finished with value: 12.749016594912073 and parameters: {'entry_z': 1.1900710349518113, 'exit_z': 0.04988898877399477}. Best is trial 831 with value: 14.236793151228433.
[I 2025-09-29 11:43:10,825] Trial 1166 finished with value: 12.748011225969098 and parameters: {'entry_z': 1.0796337347450364, 'exit_z': 0.0004794116844448054}. Best is trial 831 with value: 14.236793151228433.
[I 2025-09-29 11:43:11,049] Trial 1167 finished with value: 11.680645588209638 and parameters: {'entry_z': 1.1366485386171117, 'exit_z': 0.14695855477863184}. Best is trial 831 with value: 14.236793151228433.
[I 2025-09-29 11:43:11,273] Trial 1168 finished with value: 10.791297483135208 and parameters: {'entry_z': 1.055058398495551, 'exit_z': 0.0873631477240624}. Best is trial 831 with value: 14.236793151228433.
[I 2025-09-29 11:43:11,496] Trial 1169 finished with value: 12.39648365850085 and parameters: {'entry_z': 1.2374429173879322, 'exit_z': 0.04629941302488086}. Best i

,window,best_entry_z,best_exit_z,train_trades,train_pnl,train_sharpe,train_max_dd,train_winrate,test_trades,test_pnl,test_sharpe,test_max_dd,test_winrate,train_score,test_score
0,30,1.306332,0.027809,575,24.486365,4.109486,-0.774342,0.791304,1882,69.549429,3.664836,-1.430482,0.779490,10.264628,25.705380
1,120,1.334943,0.035380,398,25.544623,5.277919,-1.930191,0.881910,1246,70.763922,4.249502,-1.253624,0.880417,10.929644,26.428924
2,375,1.113890,0.049326,255,33.801135,5.824188,-1.072444,0.862745,779,59.110917,5.544600,-0.870525,0.880616,14.236793,23.009786


### Gradient-Descent (SPSA) Optimization of Entry/Exit z (25% Train / 75% Test)

**Goal**  
Learn `entry_z` and `exit_z` per `window ∈ {30,120,375}` using **SPSA** (a fast, noisy gradient method) to maximize:
$$
\text{score} = w_{\text{pnl}}\cdot \text{PnL}
+ w_{\text{sharpe}}\cdot \text{Sharpe}
- w_{\text{dd}}\cdot \lvert \text{MaxDD} \rvert
$$

---

### Workflow
1. **Preprocess**
   - `spread = banknifty - nifty`; rolling μ/σ → `zscore`.
   - `PLValue = spread * (TTE ** 0.7)`.
   - Drop NaNs.

2. **Split**
   - Per `window`, first **25%** → **train**, last **75%** → **test**.

3. **Optimize (Train) — SPSA**
   - Start from random `(entry_z, exit_z)` within bounds.
   - At each iter `k`, evaluate score at **θ±ckΔ** and estimate gradient.
   - **Ascent update:** θ ← Proj(θ + ak·ĝ); projection enforces bounds.
   - Repeat for `gd_iters`, with `restarts` multi-start runs.
   - Keep the best train score/params.

4. **Evaluate (Test)**
   - Re-run simulator on test with the **best train params**.

---

### Trading Logic (Crossing; no double-trades)
- **Enter LONG** when z crosses **down** through `-entry_z`.
- **Enter SHORT** when z crosses **up** through `+entry_z`.
- **Exit toward 0:** Long exits at `-exit_z` (cross up), Short exits at `+exit_z` (cross down).
- Respect `min_horizon` / `max_horizon`; skip exit on entry bar; optional cooldown.

---

### Output
- `summary_gd`: per window, best params, **train** and **test** metrics + scores.
- `all_trades_test[window]`: test trades for deeper inspection.

> Tune speed/robustness via: `gd_iters`, `restarts`, step schedules (`a0, c0, alpha, gamma`), and weights `w_pnl, w_sharpe, w_dd`.

In [16]:
# ============================================================
# GRADIENT-DESCENT (SPSA) OPTIMIZATION OF ENTRY_Z / EXIT_Z
# 25% train / 75% test, windows = [30, 120, 375]
# Objective: score = w_pnl * PnL + w_sharpe * Sharpe - w_dd * |MaxDD|
# ============================================================
import numpy as np
import pandas as pd

# -----------------------------
# EDITABLE SETTINGS
# -----------------------------
windows        = [30, 120, 375]
train_frac     = 0.25

# Objective weights
w_pnl          = 1.0
w_sharpe       = 0.35
w_dd           = 0.20     # penalty on |max drawdown|

# Bounds (projected updates)
entry_bounds   = (1.0, 3.5)
exit_bounds    = (0.0, 1.5)

# Trading engine controls
min_horizon    = 30
max_horizon    = 5 * 375
cooldown_bars  = 0
roundtrip_cost = 0.0       # no cost per requirements

# SPSA hyperparameters (gradient-ascent; increase iters for deeper search)
gd_iters       = 200       # iterations per restart per window
restarts       = 3         # multi-start to avoid local optima
seed           = 42

# Step schedules a_k, c_k: a_k = a0 / (k+1)^alpha, c_k = c0 / (k+1)^gamma
a0, alpha      = 0.10, 0.602
c0, gamma      = 0.10, 0.101

rng = np.random.default_rng(seed)

# -----------------------------
# DATA PREP (expects df loaded)
# -----------------------------
df = df.sort_values('time').drop_duplicates('time').copy()
df['time']      = pd.to_datetime(df['time'])
df['banknifty'] = df['banknifty'].astype(float)
df['nifty']     = df['nifty'].astype(float)
df['tte']       = df['tte'].astype(float)

# -----------------------------
# HELPERS
# -----------------------------
def make_df_with_z(df0, window):
    d = df0.copy()
    d['spread'] = d['banknifty'] - d['nifty']
    roll = d['spread'].rolling(window, min_periods=window)
    mu   = roll.mean()
    sd   = roll.std(ddof=0).replace(0, np.nan)
    d['zscore']  = (d['spread'] - mu) / sd
    d['PLValue'] = d['spread'] * (d['tte'] ** 0.7)
    d = d.dropna(subset=['zscore','tte']).reset_index(drop=True)
    return d

def run_trading_sim(d, entry_z, exit_z,
                    min_horizon=30, max_horizon=1950, cooldown=0, cost=0.0):
    """Crossing entries; exit toward 0; skip exit on entry bar; cooldown prevents flicker."""
    trades, pos, cooldown_left = [], None, 0
    for i in range(1, len(d)):
        z_prev, z = d['zscore'].iat[i-1], d['zscore'].iat[i]

        if cooldown_left > 0:
            cooldown_left -= 1

        # ENTRY
        if pos is None and cooldown_left == 0:
            if (z_prev <=  entry_z) and (z >  entry_z):
                pos = {'side':'short', 'i0':i}; continue
            if (z_prev >= -entry_z) and (z < -entry_z):
                pos = {'side':'long',  'i0':i}; continue

        # EXIT
        if pos is not None:
            hold = i - pos['i0']
            exit_long  = (pos['side']=='long'  and (z_prev < -exit_z) and (z >= -exit_z))
            exit_short = (pos['side']=='short' and (z_prev >  exit_z) and (z <=  exit_z))
            time_up    = hold >= max_horizon

            if hold >= min_horizon and (exit_long or exit_short or time_up):
                ent = d.iloc[pos['i0']]; cur = d.iloc[i]
                pnl = (cur['PLValue'] - ent['PLValue']) if pos['side']=='long' else (ent['PLValue'] - cur['PLValue'])
                pnl -= cost
                trades.append({'pnl': float(pnl)})
                pos = None; cooldown_left = cooldown
    return pd.DataFrame(trades)

def evaluate(trades_df):
    if trades_df.empty:
        return {'trades':0, 'pnl':0.0, 'sharpe':np.nan, 'max_dd':np.nan, 'winrate':np.nan}
    pnl_curve = trades_df['pnl'].cumsum()
    rets = trades_df['pnl']
    sharpe = (rets.mean()/rets.std(ddof=0)*np.sqrt(252)) if rets.std(ddof=0)>0 else np.nan
    peak = pnl_curve.cummax()
    max_dd = (pnl_curve - peak).min()
    wins = (trades_df['pnl'] > 0).sum()
    losses = (trades_df['pnl'] < 0).sum()
    winrate = wins/(wins+losses) if (wins+losses)>0 else np.nan
    return {'trades':len(trades_df), 'pnl':float(pnl_curve.iloc[-1]),
            'sharpe':float(sharpe) if np.isfinite(sharpe) else np.nan,
            'max_dd':float(max_dd) if np.isfinite(max_dd) else np.nan,
            'winrate':float(winrate) if np.isfinite(winrate) else np.nan}

def score_from_metrics(m):
    dd_mag = abs(m['max_dd']) if np.isfinite(m['max_dd']) else 1e6
    sharpe_term = m['sharpe'] if np.isfinite(m['sharpe']) else 0.0
    return (w_pnl * m['pnl']) + (w_sharpe * sharpe_term) - (w_dd * dd_mag)

def proj(theta):
    """Project [entry_z, exit_z] into bounds."""
    ez = np.clip(theta[0], *entry_bounds)
    xz = np.clip(theta[1], *exit_bounds)
    return np.array([ez, xz], dtype=float)

def spsa_optimize(d_train, iters=200, starts=3):
    """SPSA gradient-ascent on score(entry_z, exit_z) over train slice."""
    best = {'theta': None, 'score': -np.inf}
    for s in range(starts):
        # random start within bounds
        theta = np.array([
            rng.uniform(*entry_bounds),
            rng.uniform(*exit_bounds)
        ], dtype=float)

        # keep track of best within this start
        best_local = {'theta': theta.copy(), 'score': -np.inf}

        for k in range(iters):
            ak = a0 / ((k+1) ** alpha)   # step size
            ck = c0 / ((k+1) ** gamma)   # perturbation radius

            # Bernoulli ±1 perturbation
            delta = rng.choice([-1.0, 1.0], size=2)

            theta_plus  = proj(theta + ck * delta)
            theta_minus = proj(theta - ck * delta)

            # Evaluate scores at ±
            tr_p = run_trading_sim(d_train, theta_plus[0],  theta_plus[1],
                                   min_horizon, max_horizon, cooldown_bars, roundtrip_cost)
            tr_m = run_trading_sim(d_train, theta_minus[0], theta_minus[1],
                                   min_horizon, max_horizon, cooldown_bars, roundtrip_cost)
            s_p  = score_from_metrics(evaluate(tr_p))
            s_m  = score_from_metrics(evaluate(tr_m))

            # SPSA gradient estimate (ascent)
            ghat = (s_p - s_m) / (2.0 * ck) * (1.0 / delta)

            # Update & project
            theta = proj(theta + ak * ghat)

            # Track best
            if s_p > best_local['score']:
                best_local = {'theta': theta_plus.copy(), 'score': s_p}
            if s_m > best_local['score']:
                best_local = {'theta': theta_minus.copy(), 'score': s_m}

        # compare starts
        if best_local['score'] > best['score']:
            best = best_local

    return best['theta'][0], best['theta'][1], best['score']

# -----------------------------
# RUN PER WINDOW
# -----------------------------
rows = []
all_trades_test = {}

for W in windows:
    dW = make_df_with_z(df, W)
    if dW.empty:
        rows.append({'window':W, 'msg':'insufficient data after zscore drop'})
        continue

    split = int(train_frac * len(dW))
    d_train = dW.iloc[:split].reset_index(drop=True)
    d_test  = dW.iloc[split:].reset_index(drop=True)

    # Optimize entry/exit on TRAIN via SPSA (gradient-ascent)
    best_entry, best_exit, train_score = spsa_optimize(d_train, iters=gd_iters, starts=restarts)

    # Evaluate on TRAIN & TEST
    tr_train = run_trading_sim(d_train, best_entry, best_exit,
                               min_horizon, max_horizon, cooldown_bars, roundtrip_cost)
    met_train = evaluate(tr_train)

    tr_test = run_trading_sim(d_test, best_entry, best_exit,
                              min_horizon, max_horizon, cooldown_bars, roundtrip_cost)
    met_test = evaluate(tr_test)

    rows.append({
        'window': W,
        'best_entry_z': float(best_entry),
        'best_exit_z':  float(best_exit),
        'train_trades': met_train['trades'],
        'train_pnl':    met_train['pnl'],
        'train_sharpe': met_train['sharpe'],
        'train_max_dd': met_train['max_dd'],
        'train_winrate':met_train['winrate'],
        'train_score':  float(train_score),
        'test_trades':  met_test['trades'],
        'test_pnl':     met_test['pnl'],
        'test_sharpe':  met_test['sharpe'],
        'test_max_dd':  met_test['max_dd'],
        'test_winrate': met_test['winrate'],
        'test_score':   float(score_from_metrics(met_test))
    })
    all_trades_test[W] = tr_test

summary_gd = pd.DataFrame(rows).sort_values('window').reset_index(drop=True)
summary_gd

,window,best_entry_z,best_exit_z,train_trades,train_pnl,train_sharpe,train_max_dd,train_winrate,train_score,test_trades,test_pnl,test_sharpe,test_max_dd,test_winrate,test_score
0,30,1.436299,0.000000,557,23.573895,3.798946,-1.379118,0.789946,24.627702,1827,68.533216,3.554301,-1.791438,0.776136,69.418933
1,120,1.367674,0.070466,391,24.023595,5.231689,-1.930191,0.879795,25.468648,1227,73.060037,4.463632,-1.072444,0.877751,74.407820
2,375,1.112541,0.000000,252,33.578357,5.824150,-1.072444,0.865079,35.402321,764,59.825462,5.653963,-0.870525,0.883508,61.630244


### Coordinate Descent for z-Score Entry/Exit (25% Train / 75% Test)

**Goal**  
Find `entry_z` and `exit_z` per `window ∈ {30,120,375}` that maximize a **Calmar-boosted Sharpe** score:

$$
\text{Score} \;=\; \text{Sharpe} \;+\; \lambda \cdot \frac{\text{PnL}}{\epsilon + \lvert \text{MaxDD} \rvert}
$$

---

### Workflow
1. **Prep**
   - `spread = banknifty - nifty`
   - Rolling μ/σ → `zscore = (spread - μ)/σ`
   - `PLValue = spread \cdot (\text{TTE}^{0.7})`
   - Drop rows with NaNs.

2. **Split**
   - For each `window`, first **25%** → **train**, last **75%** → **test**.

3. **Optimize (Train) — Coordinate Descent**
   - Start at `entry_z = 1.5`, `exit_z = 0.5`.
   - Alternately nudge `entry_z` (±step) then `exit_z` (±step).
   - Keep changes that **increase the score**; shrink steps if no improvement.
   - Respect bounds: `entry_z ∈ [1.0, 3.5]`, `exit_z ∈ [0.0, 1.5]`.

4. **Evaluate (Test)**
   - Re-run simulator on test with best train params.

---

### Trading Logic (Crossing; flicker-safe)
- **Enter LONG** when z crosses **down** through `-entry_z`.  
- **Enter SHORT** when z crosses **up** through `+entry_z`.  
- **Exit toward 0**: Long exits at `-exit_z` (cross up), Short exits at `+exit_z` (cross down).  
- Enforce `min_horizon` / `max_horizon`; skip exit on entry bar; optional `cooldown_bars`.

---

### Outputs
- `summary_simple`: per window → best `entry_z`, `exit_z`, train/test metrics (**PnL, Sharpe, MaxDD, Winrate**) and **Score**.
- `test_trades_by_window[window]`: test trades for inspection.

**Tips:** Increase `passes` or step sizes to search wider; raise `λ` (`lam_calmar`) to favor higher PnL per drawdown; add `cooldown_bars=1–3` if you see churn.

In [18]:
# ==============================================================
# Optuna tuning of entry/exit z using Calmar-boosted Sharpe score
# Windows = [30, 120, 375], Split = 25% train / 75% test
# ==============================================================
import numpy as np
import pandas as pd
import optuna

# -------------------------
# EDITABLE SETTINGS
# -------------------------
windows          = [30, 120, 375]
train_frac       = 0.25

# Calmar-boosted Sharpe:  score = Sharpe + lam * (PnL / (eps + |MaxDD|))
lam_calmar       = 0.7    # ↑ more weight -> favors PnL per drawdown
eps_dd           = 1e-6

# Search bounds for thresholds
entry_z_min, entry_z_max = 1.0, 3.5
exit_z_min,  exit_z_max  = 0.0, 1.5

# Trading engine controls
min_horizon      = 30
max_horizon      = 5 * 375
cooldown_bars    = 0       # crossing + skip-exit-on-entry avoids double trades
roundtrip_cost   = 0.0     # as requested (no costs)

# Optuna
n_trials         = 500     # increase to 300–500 if you want deeper search
seed             = 42

# -------------------------
# PREP (expects df with columns: time, banknifty, nifty, tte)
# -------------------------
df = df.sort_values('time').drop_duplicates('time').copy()
df['time']      = pd.to_datetime(df['time'])
df['banknifty'] = df['banknifty'].astype(float)
df['nifty']     = df['nifty'].astype(float)
df['tte']       = df['tte'].astype(float)

# -------------------------
# HELPERS
# -------------------------
def make_df_with_z(df0, window):
    d = df0.copy()
    d['spread']  = d['banknifty'] - d['nifty']
    roll         = d['spread'].rolling(window, min_periods=window)
    mu           = roll.mean()
    sd           = roll.std(ddof=0).replace(0, np.nan)
    d['zscore']  = (d['spread'] - mu) / sd
    d['PLValue'] = d['spread'] * (d['tte'] ** 0.7)
    d = d.dropna(subset=['zscore', 'tte']).reset_index(drop=True)
    return d

def run_trading_sim(d, entry_z, exit_z,
                    min_horizon=30, max_horizon=1950, cooldown=0, cost=0.0):
    """
    Entry: CROSS up through +entry_z -> SHORT; CROSS down through -entry_z -> LONG
    Exit:  toward 0 via CROSS to ±exit_z; time stop at max_horizon
    Skip exit checks on the entry bar; optional cooldown after exit.
    """
    trades, pos, cooldown_left = [], None, 0
    for i in range(1, len(d)):
        z_prev, z = d['zscore'].iat[i-1], d['zscore'].iat[i]

        if cooldown_left > 0:
            cooldown_left -= 1

        # ENTRY (crossing)
        if pos is None and cooldown_left == 0:
            if (z_prev <=  entry_z) and (z >  entry_z):
                pos = {'side': 'short', 'i0': i}
                continue  # important: skip exit on same bar
            if (z_prev >= -entry_z) and (z < -entry_z):
                pos = {'side': 'long', 'i0': i}
                continue

        # EXIT (toward zero)
        if pos is not None:
            hold = i - pos['i0']
            exit_long  = (pos['side']=='long'  and (z_prev < -exit_z) and (z >= -exit_z))
            exit_short = (pos['side']=='short' and (z_prev >  exit_z) and (z <=  exit_z))
            time_up    = hold >= max_horizon

            if hold >= min_horizon and (exit_long or exit_short or time_up):
                ent = d.iloc[pos['i0']]
                cur = d.iloc[i]
                pnl = (cur['PLValue'] - ent['PLValue']) if pos['side']=='long' else (ent['PLValue'] - cur['PLValue'])
                pnl -= cost
                trades.append({'pnl': float(pnl)})
                pos = None
                cooldown_left = cooldown
    return pd.DataFrame(trades)

def evaluate(trades_df):
    if trades_df.empty:
        return {'trades':0, 'pnl':0.0, 'sharpe':np.nan, 'max_dd':np.nan, 'winrate':np.nan}
    pnl_curve = trades_df['pnl'].cumsum()
    rets      = trades_df['pnl']
    sharpe    = (rets.mean()/rets.std(ddof=0)*np.sqrt(252)) if rets.std(ddof=0)>0 else np.nan
    peak      = pnl_curve.cummax()
    max_dd    = (pnl_curve - peak).min()
    wins      = (trades_df['pnl'] > 0).sum()
    losses    = (trades_df['pnl'] < 0).sum()
    winrate   = wins/(wins+losses) if (wins+losses)>0 else np.nan
    return {
        'trades': len(trades_df),
        'pnl': float(pnl_curve.iloc[-1]),
        'sharpe': float(sharpe) if np.isfinite(sharpe) else np.nan,
        'max_dd': float(max_dd) if np.isfinite(max_dd) else np.nan,
        'winrate': float(winrate) if np.isfinite(winrate) else np.nan
    }

def score_calmar_sharpe(m, lam=0.5, eps=1e-6):
    """Sharpe + lam * Calmar, where Calmar = PnL / (eps + |DD|)"""
    sharpe = m['sharpe'] if np.isfinite(m['sharpe']) else 0.0
    dd_mag = abs(m['max_dd']) if np.isfinite(m['max_dd']) else np.inf
    calmar = m['pnl'] / (eps + dd_mag) if np.isfinite(dd_mag) else 0.0
    return sharpe + lam * calmar

# -------------------------
# RUN PER WINDOW
# -------------------------
rows = []
test_trades_by_window = {}

for W in windows:
    dW = make_df_with_z(df, W)
    if dW.empty:
        rows.append({'window': W, 'msg': 'insufficient data after z-score drop'})
        continue

    split = int(train_frac * len(dW))
    d_train = dW.iloc[:split].reset_index(drop=True)
    d_test  = dW.iloc[split:].reset_index(drop=True)

    # Optuna objective on TRAIN: maximize Calmar-boosted Sharpe
    def objective(trial):
        ez = trial.suggest_float('entry_z', entry_z_min, entry_z_max)
        xz = trial.suggest_float('exit_z',  exit_z_min,  exit_z_max)
        tr = run_trading_sim(d_train, ez, xz,
                             min_horizon=min_horizon, max_horizon=max_horizon,
                             cooldown=cooldown_bars, cost=roundtrip_cost)
        met = evaluate(tr)
        return score_calmar_sharpe(met, lam=lam_calmar, eps=eps_dd)

    study = optuna.create_study(direction='maximize',
                                sampler=optuna.samplers.TPESampler(seed=seed))
    study.optimize(objective, n_trials=n_trials, show_progress_bar=False)

    best_ez = float(study.best_trial.params['entry_z'])
    best_xz = float(study.best_trial.params['exit_z'])

    # Evaluate on TRAIN & TEST for reporting
    tr_train = run_trading_sim(d_train, best_ez, best_xz,
                               min_horizon=min_horizon, max_horizon=max_horizon,
                               cooldown=cooldown_bars, cost=roundtrip_cost)
    met_train = evaluate(tr_train)
    train_score = score_calmar_sharpe(met_train, lam=lam_calmar, eps=eps_dd)

    tr_test = run_trading_sim(d_test, best_ez, best_xz,
                              min_horizon=min_horizon, max_horizon=max_horizon,
                              cooldown=cooldown_bars, cost=roundtrip_cost)
    met_test = evaluate(tr_test)
    test_score = score_calmar_sharpe(met_test, lam=lam_calmar, eps=eps_dd)

    rows.append({
        'window': W,
        'best_entry_z': best_ez,
        'best_exit_z':  best_xz,
        'train_trades': met_train['trades'],
        'train_pnl':    met_train['pnl'],
        'train_sharpe': met_train['sharpe'],
        'train_max_dd': met_train['max_dd'],
        'train_winrate':met_train['winrate'],
        'train_score':  float(train_score),
        'test_trades':  met_test['trades'],
        'test_pnl':     met_test['pnl'],
        'test_sharpe':  met_test['sharpe'],
        'test_max_dd':  met_test['max_dd'],
        'test_winrate': met_test['winrate'],
        'test_score':   float(test_score)
    })
    test_trades_by_window[W] = tr_test

summary_calmar = pd.DataFrame(rows).sort_values('window').reset_index(drop=True)
summary_calmar

[I 2025-09-29 12:07:08,287] A new study created in memory with name: no-name-1c36b710-05eb-484c-a2da-0d31f1908c9d
[I 2025-09-29 12:07:08,513] Trial 0 finished with value: 6.995441679562939 and parameters: {'entry_z': 1.9363502971184063, 'exit_z': 1.4260714596148742}. Best is trial 0 with value: 6.995441679562939.
[I 2025-09-29 12:07:08,712] Trial 1 finished with value: 41.37489894825564 and parameters: {'entry_z': 2.8299848545285125, 'exit_z': 0.8979877262955549}. Best is trial 1 with value: 41.37489894825564.
[I 2025-09-29 12:07:08,949] Trial 2 finished with value: 14.028259049795942 and parameters: {'entry_z': 1.3900466011060912, 'exit_z': 0.23399178050430397}. Best is trial 1 with value: 41.37489894825564.
[I 2025-09-29 12:07:09,180] Trial 3 finished with value: 10.001428414790242 and parameters: {'entry_z': 1.1452090304204987, 'exit_z': 1.2992642186624028}. Best is trial 1 with value: 41.37489894825564.
[I 2025-09-29 12:07:09,399] Trial 4 finished with value: 49.00970021597617 and 

[I 2025-09-29 12:07:17,064] Trial 40 finished with value: 44.776742917707395 and parameters: {'entry_z': 2.7671936835901993, 'exit_z': 0.5123251424032387}. Best is trial 35 with value: 52.00649824744474.
[I 2025-09-29 12:07:17,289] Trial 41 finished with value: 56.51969972733406 and parameters: {'entry_z': 2.550372269461394, 'exit_z': 0.8379083181896015}. Best is trial 41 with value: 56.51969972733406.
[I 2025-09-29 12:07:17,499] Trial 42 finished with value: 36.00883514498001 and parameters: {'entry_z': 2.6290192040476144, 'exit_z': 0.8373447428953512}. Best is trial 41 with value: 56.51969972733406.
[I 2025-09-29 12:07:17,717] Trial 43 finished with value: 15.783497908324634 and parameters: {'entry_z': 2.335905449402149, 'exit_z': 0.7115663433353123}. Best is trial 41 with value: 56.51969972733406.
[I 2025-09-29 12:07:17,916] Trial 44 finished with value: 40.91591056586603 and parameters: {'entry_z': 3.004511477684192, 'exit_z': 0.9019179647248579}. Best is trial 41 with value: 56.51

[I 2025-09-29 12:07:25,872] Trial 81 finished with value: 61.1974751222151 and parameters: {'entry_z': 2.5851069759485954, 'exit_z': 0.8208988117155221}. Best is trial 74 with value: 107.08949798399485.
[I 2025-09-29 12:07:26,090] Trial 82 finished with value: 45.106806358830674 and parameters: {'entry_z': 2.713305271534566, 'exit_z': 0.8060029505132292}. Best is trial 74 with value: 107.08949798399485.
[I 2025-09-29 12:07:26,301] Trial 83 finished with value: 35.61125783053294 and parameters: {'entry_z': 2.6152129339653802, 'exit_z': 0.727228351032813}. Best is trial 74 with value: 107.08949798399485.
[I 2025-09-29 12:07:26,517] Trial 84 finished with value: 59.362892647446586 and parameters: {'entry_z': 2.484723171034164, 'exit_z': 0.8213914728377624}. Best is trial 74 with value: 107.08949798399485.
[I 2025-09-29 12:07:26,718] Trial 85 finished with value: 44.263083554882414 and parameters: {'entry_z': 2.8837156518479907, 'exit_z': 0.6182098144774891}. Best is trial 74 with value: 1

[I 2025-09-29 12:07:34,611] Trial 122 finished with value: 34.615025116371505 and parameters: {'entry_z': 2.6624868731800397, 'exit_z': 0.6869186631273453}. Best is trial 102 with value: 118.50784230216857.
[I 2025-09-29 12:07:34,824] Trial 123 finished with value: 96.8799952802084 and parameters: {'entry_z': 2.566171512717663, 'exit_z': 0.7230364277761835}. Best is trial 102 with value: 118.50784230216857.
[I 2025-09-29 12:07:35,034] Trial 124 finished with value: 43.858868799018765 and parameters: {'entry_z': 2.782941270281224, 'exit_z': 0.5960885369467747}. Best is trial 102 with value: 118.50784230216857.
[I 2025-09-29 12:07:35,259] Trial 125 finished with value: 9.609968330402737 and parameters: {'entry_z': 2.3050756213319064, 'exit_z': 0.749983293853325}. Best is trial 102 with value: 118.50784230216857.
[I 2025-09-29 12:07:35,467] Trial 126 finished with value: 47.25490810366037 and parameters: {'entry_z': 2.7094832411374012, 'exit_z': 0.6839632192097023}. Best is trial 102 with

[I 2025-09-29 12:07:43,318] Trial 162 finished with value: 97.58107456294954 and parameters: {'entry_z': 2.538385230636921, 'exit_z': 0.7029961613423849}. Best is trial 102 with value: 118.50784230216857.
[I 2025-09-29 12:07:43,528] Trial 163 finished with value: 35.440936199206284 and parameters: {'entry_z': 2.6612583900933022, 'exit_z': 0.7649654252158409}. Best is trial 102 with value: 118.50784230216857.
[I 2025-09-29 12:07:43,738] Trial 164 finished with value: 104.8479549561679 and parameters: {'entry_z': 2.601326365535418, 'exit_z': 0.7353612400404459}. Best is trial 102 with value: 118.50784230216857.
[I 2025-09-29 12:07:43,945] Trial 165 finished with value: 44.569481891399114 and parameters: {'entry_z': 2.759666350552311, 'exit_z': 0.7360286664714508}. Best is trial 102 with value: 118.50784230216857.
[I 2025-09-29 12:07:44,159] Trial 166 finished with value: 99.0512959531002 and parameters: {'entry_z': 2.6062276478341873, 'exit_z': 0.6441704853352324}. Best is trial 102 with

...

[I 2025-09-29 12:11:43,946] Trial 339 finished with value: 23.268020295235914 and parameters: {'entry_z': 3.0731056068375144, 'exit_z': 0.8801515412508996}. Best is trial 83 with value: 23.76379973988033.
[I 2025-09-29 12:11:44,143] Trial 340 finished with value: 13.113946831881071 and parameters: {'entry_z': 3.0206548593037725, 'exit_z': 0.2800493912244207}. Best is trial 83 with value: 23.76379973988033.
[I 2025-09-29 12:11:44,361] Trial 341 finished with value: 10.658974857464123 and parameters: {'entry_z': 1.270765614804587, 'exit_z': 0.8198836835535935}. Best is trial 83 with value: 23.76379973988033.
[I 2025-09-29 12:11:44,556] Trial 342 finished with value: 12.075938417471919 and parameters: {'entry_z': 3.2125964326305994, 'exit_z': 0.7655340244024091}. Best is trial 83 with value: 23.76379973988033.
[I 2025-09-29 12:11:44,752] Trial 343 finished with value: 23.159970859229677 and parameters: {'entry_z': 3.1180434579365803, 'exit_z': 0.8568972737782604}. Best is trial 83 with va

[I 2025-09-29 12:11:52,022] Trial 380 finished with value: 22.516863351099172 and parameters: {'entry_z': 3.0563869051871695, 'exit_z': 0.8486461691725065}. Best is trial 83 with value: 23.76379973988033.
[I 2025-09-29 12:11:52,216] Trial 381 finished with value: 23.30118095921135 and parameters: {'entry_z': 3.0974688230141436, 'exit_z': 0.8899640936603987}. Best is trial 83 with value: 23.76379973988033.
[I 2025-09-29 12:11:52,410] Trial 382 finished with value: 12.718035722661861 and parameters: {'entry_z': 3.173931222464924, 'exit_z': 0.7428475495632626}. Best is trial 83 with value: 23.76379973988033.
[I 2025-09-29 12:11:52,607] Trial 383 finished with value: 23.245604592435033 and parameters: {'entry_z': 2.939968716872953, 'exit_z': 0.8212590473355972}. Best is trial 83 with value: 23.76379973988033.
[I 2025-09-29 12:11:52,803] Trial 384 finished with value: 14.57038989469617 and parameters: {'entry_z': 3.030105281304495, 'exit_z': 0.7969944873319894}. Best is trial 83 with value:

[I 2025-09-29 12:12:00,058] Trial 421 finished with value: 12.383921662030822 and parameters: {'entry_z': 3.1692088094989446, 'exit_z': 0.7027975162589646}. Best is trial 83 with value: 23.76379973988033.
[I 2025-09-29 12:12:00,254] Trial 422 finished with value: 14.45417747747572 and parameters: {'entry_z': 3.0021451624662086, 'exit_z': 0.8161280158409122}. Best is trial 83 with value: 23.76379973988033.
[I 2025-09-29 12:12:00,446] Trial 423 finished with value: 13.499296693171068 and parameters: {'entry_z': 3.130696525578702, 'exit_z': 0.6437024488882789}. Best is trial 83 with value: 23.76379973988033.
[I 2025-09-29 12:12:00,639] Trial 424 finished with value: 21.74139888537247 and parameters: {'entry_z': 3.0627158240801458, 'exit_z': 0.8642077823257142}. Best is trial 83 with value: 23.76379973988033.
[I 2025-09-29 12:12:00,834] Trial 425 finished with value: 20.180449201446653 and parameters: {'entry_z': 3.188346580339483, 'exit_z': 0.8189949252725294}. Best is trial 83 with value

[I 2025-09-29 12:12:08,112] Trial 462 finished with value: 13.305750514515964 and parameters: {'entry_z': 3.0949483219062075, 'exit_z': 0.209795022845227}. Best is trial 83 with value: 23.76379973988033.
[I 2025-09-29 12:12:08,309] Trial 463 finished with value: 14.463698346959829 and parameters: {'entry_z': 3.012080644773668, 'exit_z': 0.7989765904034383}. Best is trial 83 with value: 23.76379973988033.
[I 2025-09-29 12:12:08,506] Trial 464 finished with value: 17.66511054359433 and parameters: {'entry_z': 3.1974461811722232, 'exit_z': 0.9006972671655753}. Best is trial 83 with value: 23.76379973988033.
[I 2025-09-29 12:12:08,701] Trial 465 finished with value: 22.249658699980234 and parameters: {'entry_z': 3.129524559319627, 'exit_z': 0.8338442065510043}. Best is trial 83 with value: 23.76379973988033.
[I 2025-09-29 12:12:08,896] Trial 466 finished with value: 14.594927600857284 and parameters: {'entry_z': 3.0580890278268344, 'exit_z': 0.8175328705996027}. Best is trial 83 with value

,window,best_entry_z,best_exit_z,train_trades,train_pnl,train_sharpe,train_max_dd,train_winrate,train_score,test_trades,test_pnl,test_sharpe,test_max_dd,test_winrate,test_score
0,30,2.588191,0.770274,287,12.940281,6.240130,-0.080683,0.804878,118.507842,941,32.503629,3.568575,-0.960510,0.743889,27.256520
1,120,2.716313,1.499142,152,8.908882,5.732365,-0.169071,0.776316,42.617335,451,17.970592,3.370939,-0.925729,0.751663,16.959579
2,375,3.093479,0.834692,74,9.383602,7.542153,-0.404922,0.878378,23.763800,224,16.801396,5.346454,-0.820157,0.847534,19.686340


### Advanced ML-Based Adaptive Parameter Prediction

**Goal**  
Predict **window**, **entry_z**, **exit_z**, **min/max horizon**, **position size**, and **regime** from rich features, then trade with those parameters on rolling test windows.

---

### Pipeline
1. **Prep**
   - `spread = banknifty - nifty`
   - Features: 40+ signals (spread stats, vol ratios, regime flags, half-life proxies, RSI, seasonality, autocorr, time-to-expiry effects, trend strength, microstructure proxies).

2. **Training (first 50% of data)**
   - Build features via `create_comprehensive_features`.
   - For multiple historical periods:
     - **Optimize** simple trading params (window, entry_z, exit_z, horizons) with an **enhanced grid** → create labels (best params).
     - **Detect regime** (low/high vol) from features.
     - Collect many (features → labels, regime) samples across the period.
   - Train:
     - **Regime classifier** (`GradientBoostingClassifier`).
     - **Regressors** (`RandomForestRegressor`) for: window, entry_z, exit_z, min/max horizon, position size.
   - Standardize inputs with `StandardScaler`.

3. **Prediction & Trading (remaining 50%)**
   - Walk forward in blocks (`rebalance_freq` rows).
   - Create features for the block; predict:
     - **Regime**, **window**, **entry_z**, **exit_z**, **min_horizon**, **max_horizon**, **position_size** (with smart bounds by regime).
   - Build z-score series with predicted **window**; trade using **crossing logic**:
     - **Enter LONG** when z crosses **down** through `-entry_z`.
     - **Enter SHORT** when z crosses **up** through `+entry_z`.
     - **Exit toward 0**: Long at `-exit_z` (cross up), Short at `+exit_z` (cross down).
     - Enforce min/max horizon, skip exit on entry bar; optional cooldown.
   - Scale PnL by **position_size**.

---

### Metrics
- Per block & overall: **PnL**, **Sharpe**, **Max Drawdown**, **Win rate**, **Trades**.

---

### Notes
- Robust to regime changes via regime-conditioned bounds and feature-rich predictors.
- If training fails or features insufficient, handles gracefully.
- Tune `rebalance_freq`, model depths, and the inner parameter search in `optimize_parameters_for_period` for speed vs. performance.

> Output: `final_trades` (all executed trades) and `final_metrics` (aggregate performance).

In [20]:
# ==============================================================
# ADVANCED ML-BASED ADAPTIVE PARAMETER PREDICTION
# Predicts: window, entry_z, exit_z, position_size, regime, horizons
# Features: Market microstructure, volatility regimes, time effects
# ==============================================================
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import TimeSeriesSplit
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings('ignore')

# Prepare data
df = df.sort_values('time').drop_duplicates('time').copy()
df['time'] = pd.to_datetime(df['time'])
df['spread'] = df['banknifty'] - df['nifty']

# ==============================================================
# ENHANCED FEATURE ENGINEERING
# ==============================================================
def create_comprehensive_features(df, base_window=120):
    """Create 40+ features for ML models to predict optimal parameters"""
    features = pd.DataFrame(index=df.index)
    
    # === SPREAD DYNAMICS ===
    features['spread'] = df['spread']
    features['spread_abs'] = df['spread'].abs()
    features['spread_momentum_5'] = df['spread'].diff(5)
    features['spread_momentum_20'] = df['spread'].diff(20)
    features['spread_acceleration'] = features['spread_momentum_5'].diff(5)
    
    # Rolling statistics with multiple windows
    for w in [30, 60, 120, 240]:
        roll = df['spread'].rolling(w, min_periods=max(10, w//4))
        features[f'spread_mean_{w}'] = roll.mean()
        features[f'spread_std_{w}'] = roll.std()
        features[f'spread_skew_{w}'] = roll.skew()
        features[f'spread_kurt_{w}'] = roll.kurt()
        features[f'spread_min_{w}'] = roll.min()
        features[f'spread_max_{w}'] = roll.max()
        features[f'spread_range_{w}'] = features[f'spread_max_{w}'] - features[f'spread_min_{w}']
        features[f'spread_zscore_{w}'] = (df['spread'] - features[f'spread_mean_{w}']) / (features[f'spread_std_{w}'] + 1e-8)
    
    # === VOLATILITY REGIMES ===
    features['banknifty_vol_30'] = df['banknifty'].rolling(30).std()
    features['nifty_vol_30'] = df['nifty'].rolling(30).std() 
    features['banknifty_vol_120'] = df['banknifty'].rolling(120).std()
    features['nifty_vol_120'] = df['nifty'].rolling(120).std()
    
    features['vol_ratio_30'] = features['banknifty_vol_30'] / (features['nifty_vol_30'] + 1e-8)
    features['vol_ratio_120'] = features['banknifty_vol_120'] / (features['nifty_vol_120'] + 1e-8)
    features['vol_spread_30'] = features['banknifty_vol_30'] - features['nifty_vol_30']
    features['vol_spread_120'] = features['banknifty_vol_120'] - features['nifty_vol_120']
    
    # Volatility regime detection
    features['high_vol_regime'] = (features['spread_std_120'] > 
                                  features['spread_std_120'].rolling(480).quantile(0.75)).astype(int)
    features['vol_regime_change'] = features['high_vol_regime'].diff().abs()
    
    # === MEAN REVERSION STRENGTH ===
    def half_life_estimate(series, max_lag=60):
        """Estimate mean reversion half-life"""
        try:
            if len(series) < max_lag * 2:
                return np.nan
            autocorrs = [series.autocorr(lag=i) for i in range(1, min(max_lag, len(series)//2))]
            autocorrs = [x for x in autocorrs if not pd.isna(x)]
            if not autocorrs or autocorrs[0] <= 0:
                return np.nan
            # Find where autocorr drops to 50% of initial
            target = autocorrs[0] * 0.5
            for i, ac in enumerate(autocorrs):
                if ac <= target:
                    return i + 1
            return max_lag
        except:
            return np.nan
    
    features['mean_reversion_strength'] = df['spread'].rolling(base_window).apply(
        lambda x: -np.log(abs(x.autocorr(lag=1))) if abs(x.autocorr(lag=1)) > 0.01 else 0, raw=False)
    features['half_life_estimate'] = df['spread'].rolling(base_window*2).apply(
        lambda x: half_life_estimate(x), raw=False)
    
    # === TIME-TO-EXPIRY EFFECTS ===
    features['tte'] = df['tte']
    features['tte_momentum'] = df['tte'].diff(5)
    features['tte_normalized'] = df['tte'] / (df['tte'].rolling(240).mean() + 1e-8)
    features['tte_volatility'] = df['tte'].rolling(60).std()
    
    # Time decay effects
    features['time_decay_effect'] = features['tte'] ** 0.7  # From P/L formula
    features['time_decay_momentum'] = features['time_decay_effect'].diff(5)
    
    # === MARKET MICROSTRUCTURE ===
    # Bid-ask proxy and market impact
    features['spread_impact'] = features['spread_abs'] / (features['spread_std_30'] + 1e-8)
    features['market_stress'] = (features['spread_range_30'] > 
                                features['spread_range_30'].rolling(120).quantile(0.8)).astype(int)
    
    # Price clustering and round number effects
    features['spread_decimal'] = (df['spread'] * 100) % 1  # Decimal component
    features['near_round'] = (np.abs(features['spread_decimal'] - 0.5) > 0.4).astype(int)
    
    # === SEASONALITY AND TIME EFFECTS ===
    df_time = pd.to_datetime(df['time'])
    features['hour'] = df_time.dt.hour
    features['minute'] = df_time.dt.minute
    features['day_of_week'] = df_time.dt.dayofweek
    features['time_to_close'] = np.maximum(0, 15.5 - (features['hour'] + features['minute']/60))
    
    # Market session effects
    features['market_open'] = ((features['hour'] == 9) & (features['minute'] <= 30)).astype(int)
    features['market_close'] = ((features['hour'] >= 15) & (features['minute'] >= 0)).astype(int)
    features['lunch_time'] = ((features['hour'] >= 12) & (features['hour'] <= 13)).astype(int)
    
    # === MOMENTUM AND TREND ===
    features['trend_strength_60'] = df['spread'].rolling(60).apply(
        lambda x: abs(np.corrcoef(np.arange(len(x)), x)[0,1]) if len(x) > 1 else 0, raw=False)
    features['trend_strength_240'] = df['spread'].rolling(240).apply(
        lambda x: abs(np.corrcoef(np.arange(len(x)), x)[0,1]) if len(x) > 1 else 0, raw=False)
    
    # Momentum oscillator
    features['rsi_14'] = calculate_rsi(df['spread'], 14)
    features['rsi_30'] = calculate_rsi(df['spread'], 30)
    
    # === AUTOCORRELATION FEATURES ===
    for lag in [1, 5, 10, 20]:
        features[f'spread_autocorr_{lag}'] = df['spread'].rolling(base_window).apply(
            lambda x: x.autocorr(lag=lag) if len(x) > lag + 10 else 0, raw=False)
    
    return features.fillna(method='ffill').fillna(0)

def calculate_rsi(series, window):
    """Calculate RSI momentum indicator"""
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

# ==============================================================
# ML MODEL FOR MULTI-PARAMETER PREDICTION
# ==============================================================
class AdaptiveParameterPredictor:
    def __init__(self):
        # Models for different parameter types
        self.models = {
            'window': RandomForestRegressor(n_estimators=150, max_depth=15, random_state=42),
            'entry_z': RandomForestRegressor(n_estimators=150, max_depth=15, random_state=42),  
            'exit_z': RandomForestRegressor(n_estimators=150, max_depth=15, random_state=42),
            'min_horizon': RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42),
            'max_horizon': RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42),
            'position_size': RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
        }
        
        # Regime classifier
        self.regime_classifier = GradientBoostingClassifier(n_estimators=100, random_state=42)
        
        # Scalers for features
        self.scalers = {param: StandardScaler() for param in self.models.keys()}
        self.feature_scaler = StandardScaler()
        
        self.is_fitted = False
        self.feature_importance = {}
        
    def create_training_targets(self, df_raw, n_periods=12):
        """Create training data with optimal parameters for different market periods"""
        print("Creating comprehensive training data...")
        
        total_len = len(df_raw)
        period_size = total_len // n_periods
        
        X_data, y_data, regime_data = [], [], []
        
        for i in range(n_periods - 1):
            start_idx = i * period_size
            end_idx = (i + 2) * period_size
            
            if end_idx > total_len:
                break
                
            period_data = df_raw.iloc[start_idx:end_idx].copy()
            print(f"Processing training period {i+1}/{n_periods-1}...")
            
            # Create features for this period
            features = create_comprehensive_features(period_data)
            
            # Find optimal parameters using enhanced grid search
            best_params = self.optimize_parameters_for_period(period_data)
            
            # Determine market regime using clustering
            regime = self.detect_market_regime(features, period_data)
            
            # Use multiple time points from this period
            mid_start = len(features) // 3
            mid_end = 2 * len(features) // 3
            
            for idx in range(mid_start, mid_end, 50):  # Sample every 50 rows
                if idx < len(features):
                    feature_row = features.iloc[idx]
                    X_data.append(feature_row.values)
                    y_data.append([
                        best_params['window'],
                        best_params['entry_z'],
                        best_params['exit_z'], 
                        best_params['min_horizon'],
                        best_params['max_horizon'],
                        best_params['position_size']
                    ])
                    regime_data.append(regime)
        
        return np.array(X_data), np.array(y_data), np.array(regime_data)
    
    def optimize_parameters_for_period(self, period_data):
        """Enhanced parameter optimization for a specific period"""
        best_score = -np.inf
        best_params = {
            'window': 120, 'entry_z': 2.0, 'exit_z': 0.5,
            'min_horizon': 30, 'max_horizon': 1000, 'position_size': 1.0
        }
        
        # Smart parameter search based on market conditions
        vol_level = period_data['spread'].std()
        
        # Adaptive parameter ranges based on volatility
        if vol_level > period_data['spread'].rolling(len(period_data)).std().quantile(0.75):
            # High volatility regime
            window_range = [60, 120, 180]
            entry_range = [1.5, 2.0, 2.5]
            exit_range = [0.3, 0.5, 0.7]
        else:
            # Low volatility regime  
            window_range = [120, 240, 360]
            entry_range = [2.0, 2.5, 3.0]
            exit_range = [0.5, 0.7, 1.0]
            
        for window in window_range:
            for entry_z in entry_range:
                for exit_z in exit_range:
                    try:
                        df_test = make_df_with_z(period_data, window)
                        if len(df_test) < 100:
                            continue
                            
                        trades = run_trading_sim(df_test, entry_z, exit_z, 
                                               min_horizon=20, max_horizon=1200, 
                                               cooldown=3, cost=0.0)
                        metrics = evaluate(trades)
                        
                        # Enhanced scoring function
                        score = self.calculate_enhanced_score(metrics)
                        
                        if score > best_score:
                            best_score = score
                            best_params.update({
                                'window': window,
                                'entry_z': entry_z,
                                'exit_z': exit_z,
                                'min_horizon': 20,
                                'max_horizon': 1200,
                                'position_size': min(1.5, max(0.5, 1.0 / (vol_level + 0.1)))
                            })
                    except:
                        continue
        
        return best_params
    
    def calculate_enhanced_score(self, metrics):
        """Enhanced scoring that considers multiple objectives"""
        if metrics['trades'] < 5:
            return -1000
            
        pnl = metrics['pnl'] if np.isfinite(metrics['pnl']) else -1000
        sharpe = metrics['sharpe'] if np.isfinite(metrics['sharpe']) else -10
        max_dd = metrics['max_dd'] if np.isfinite(metrics['max_dd']) else -1000
        winrate = metrics['winrate'] if np.isfinite(metrics['winrate']) else 0
        
        # Multi-objective score with adaptive weights
        calmar = pnl / (abs(max_dd) + 1e-6)
        trade_frequency_bonus = min(1.0, metrics['trades'] / 20.0)  # Prefer reasonable trade frequency
        
        return (0.3 * sharpe + 
                0.3 * calmar + 
                0.2 * winrate +
                0.1 * np.log(max(1, pnl)) +
                0.1 * trade_frequency_bonus)
    
    def detect_market_regime(self, features, period_data):
        """Detect market regime using simple clustering"""
        try:
            # Use key regime indicators
            regime_features = np.column_stack([
                features['spread_std_120'].fillna(0),
                features['mean_reversion_strength'].fillna(0),
                features['trend_strength_240'].fillna(0),
                features['high_vol_regime'].fillna(0)
            ])
            
            # Simple classification: 0 = low vol, 1 = high vol
            vol_threshold = np.nanquantile(regime_features[:, 0], 0.7)
            return 1 if np.nanmean(regime_features[:, 0]) > vol_threshold else 0
        except:
            return 0  # Default regime
    
    def train(self, df_raw):
        """Train all models"""
        X, y, regimes = self.create_training_targets(df_raw)
        
        if len(X) == 0:
            raise ValueError("No training data generated")
        
        # Scale features
        X_scaled = self.feature_scaler.fit_transform(X)
        
        # Train regime classifier
        self.regime_classifier.fit(X_scaled, regimes)
        
        # Train parameter prediction models
        param_names = ['window', 'entry_z', 'exit_z', 'min_horizon', 'max_horizon', 'position_size']
        
        for i, param in enumerate(param_names):
            y_param = y[:, i]
            self.models[param].fit(X_scaled, y_param)
            
            # Store feature importance
            if hasattr(self.models[param], 'feature_importances_'):
                self.feature_importance[param] = self.models[param].feature_importances_
        
        self.is_fitted = True
        print("All ML models trained successfully!")
        print(f"Training completed with {len(X)} samples")
        
    def predict(self, features_row):
        """Predict all parameters for current market conditions"""
        if not self.is_fitted:
            raise ValueError("Models not trained yet!")
        
        # Scale features
        X_scaled = self.feature_scaler.transform(features_row.values.reshape(1, -1))
        
        # Predict regime
        predicted_regime = self.regime_classifier.predict(X_scaled)[0]
        
        # Predict parameters
        predictions = {'regime': predicted_regime}
        
        for param in self.models.keys():
            pred = self.models[param].predict(X_scaled)[0]
            
            # Apply intelligent bounds based on regime
            if param == 'window':
                # Shorter windows in high volatility regimes
                bound_range = (30, 300) if predicted_regime == 1 else (60, 500)
                pred = int(np.clip(pred, *bound_range))
            elif param == 'entry_z':
                # Lower thresholds in high volatility regimes  
                bound_range = (1.0, 2.5) if predicted_regime == 1 else (1.5, 3.5)
                pred = np.clip(pred, *bound_range)
            elif param == 'exit_z':
                bound_range = (0.0, 1.0) if predicted_regime == 1 else (0.0, 1.5)
                pred = np.clip(pred, *bound_range)
            elif param == 'min_horizon':
                pred = int(np.clip(pred, 10, 100))
            elif param == 'max_horizon':
                pred = int(np.clip(pred, 200, 2000))
            elif param == 'position_size':
                pred = np.clip(pred, 0.5, 2.0)
                
            predictions[param] = pred
        
        return predictions

# ==============================================================
# EXECUTION - ADAPTIVE ML STRATEGY
# ==============================================================
def run_advanced_ml_strategy(df_raw, rebalance_freq=1000):
    """Run strategy with comprehensive ML parameter prediction"""
    
    # Initialize ML predictor
    predictor = AdaptiveParameterPredictor()
    
    # Train on first 50% of data
    train_size = int(len(df_raw) * 0.5)
    train_data = df_raw.iloc[:train_size]
    
    try:
        predictor.train(train_data)
        print("Advanced ML training completed")
    except Exception as e:
        print(f"ML training failed: {e}")
        return pd.DataFrame(), {}
    
    # Test on remaining data
    test_data = df_raw.iloc[train_size:]
    all_trades = []
    
    window_num = 1
    for i in range(0, len(test_data), rebalance_freq):
        current_data = test_data.iloc[i:i+rebalance_freq]
        
        if len(current_data) < 200:
            continue
            
        print(f"Processing window {window_num}: {len(current_data)} rows")
        
        # Create features and predict parameters
        try:
            features = create_comprehensive_features(current_data)
            
            if len(features) > 100:
                # Use features from multiple points in the window
                prediction_idx = len(features) // 4  # Use early features to predict
                feature_row = features.iloc[prediction_idx]
                
                # Get ML predictions
                ml_params = predictor.predict(feature_row)
                
                print(f"ML Predicted - Regime: {ml_params['regime']}, "
                      f"Window: {ml_params['window']}, Entry: {ml_params['entry_z']:.2f}, "
                      f"Exit: {ml_params['exit_z']:.2f}, PosSize: {ml_params['position_size']:.2f}")
                
                # Run trading with predicted parameters
                df_sim = make_df_with_z(current_data, ml_params['window'])
                
                if len(df_sim) > 50:
                    window_trades = run_trading_sim(
                        df_sim, 
                        ml_params['entry_z'], 
                        ml_params['exit_z'],
                        ml_params['min_horizon'], 
                        ml_params['max_horizon'], 
                        cooldown=5, 
                        cost=0.0
                    )
                    
                    if not window_trades.empty:
                        # Apply position sizing
                        window_trades['pnl'] *= ml_params['position_size']
                        window_trades['window_id'] = window_num
                        all_trades.append(window_trades)
                        
                        # Performance summary
                        metrics = evaluate(window_trades)
                        print(f"Window performance: PnL={metrics['pnl']:.2f}, "
                              f"Trades={metrics['trades']}, Winrate={metrics['winrate']:.1%}")
            
        except Exception as e:
            print(f"Error in window {window_num}: {e}")
        
        window_num += 1
    
    # Final results
    if all_trades:
        final_trades = pd.concat(all_trades, ignore_index=True)
        final_metrics = evaluate(final_trades)
        
        print("\n=== ADVANCED ML STRATEGY RESULTS ===")
        print(f"Total PnL: {final_metrics['pnl']:.4f}")
        print(f"Sharpe Ratio: {final_metrics['sharpe']:.4f}")
        print(f"Max Drawdown: {final_metrics['max_dd']:.4f}")
        print(f"Win Rate: {final_metrics['winrate']:.1%}")
        print(f"Total Trades: {final_metrics['trades']}")
        
        return final_trades, final_metrics
    else:
        print("No trades generated")
        return pd.DataFrame(), {}

# Run the advanced strategy
print("=== Running Advanced ML Parameter Prediction ===")
final_trades, final_metrics = run_advanced_ml_strategy(df_raw)


=== Running Advanced ML Parameter Prediction ===
Creating comprehensive training data...
Processing training period 1/11...
Processing training period 2/11...
Processing training period 3/11...
Processing training period 4/11...
Processing training period 5/11...
Processing training period 6/11...
Processing training period 7/11...
Processing training period 8/11...
Processing training period 9/11...
Processing training period 10/11...
Processing training period 11/11...
All ML models trained successfully!
Training completed with 803 samples
Advanced ML training completed
Processing window 1: 1000 rows
ML Predicted - Regime: 1, Window: 120, Entry: 2.00, Exit: 0.50, PosSize: 1.00
Window performance: PnL=0.14, Trades=7, Winrate=85.7%
Processing window 2: 1000 rows
ML Predicted - Regime: 1, Window: 120, Entry: 2.00, Exit: 0.50, PosSize: 1.00
Window performance: PnL=0.09, Trades=10, Winrate=90.0%
Processing window 3: 1000 rows
ML Predicted - Regime: 0, Window: 120, Entry: 2.00, Exit: 0.50,

ML Predicted - Regime: 1, Window: 120, Entry: 2.00, Exit: 0.50, PosSize: 1.00
Window performance: PnL=1.02, Trades=9, Winrate=88.9%
Processing window 49: 1000 rows
ML Predicted - Regime: 1, Window: 120, Entry: 2.00, Exit: 0.50, PosSize: 1.00
Window performance: PnL=-0.03, Trades=6, Winrate=66.7%
Processing window 50: 1000 rows
ML Predicted - Regime: 1, Window: 120, Entry: 2.00, Exit: 0.50, PosSize: 1.00
Window performance: PnL=0.09, Trades=6, Winrate=66.7%
Processing window 51: 1000 rows
ML Predicted - Regime: 1, Window: 120, Entry: 2.00, Exit: 0.50, PosSize: 1.00
Window performance: PnL=0.13, Trades=11, Winrate=90.9%
Processing window 52: 1000 rows
ML Predicted - Regime: 1, Window: 120, Entry: 2.00, Exit: 0.50, PosSize: 1.00
Window performance: PnL=0.10, Trades=7, Winrate=100.0%
Processing window 53: 1000 rows
ML Predicted - Regime: 1, Window: 120, Entry: 2.00, Exit: 0.50, PosSize: 1.00
Window performance: PnL=0.14, Trades=8, Winrate=100.0%
Processing window 54: 1000 rows
ML Predicted